 # Analisi della willingness to pay (WTP) per veicoli elettrici

In questa analisi si intende studiare quanto i consumatori siano disposti a pagare in più per l’acquisto di un veicolo elettrico (`WTP2_1`), sulla base di:

- variabili principali: 
    * atteggiamento verso l’acquisto (`ATE_1`)
    * coscienza ambientale (`AMB_tot`),
- variabile moderatrice: 
    * livello di istruzione (`G5`),
- variabili di controllo: 
    * genere (`G1`)
    * area geografica (`Area`) 
    * possesso di veicolo elettrico (`TA7`).

L’obiettivo è stimare un modello di regressione lineare in grado di spiegare la disponibilità a pagare, tenendo conto dell’effetto moderatore dell’istruzione e controllando per altre caratteristiche socio-demografiche.


## Caricamento, selezione e preparazione iniziale dei dati

Nel blocco di codice seguente si svolgono le fasi 1–3 della pipeline analitica:

1. Viene caricato il file CSV (in formato long), selezionando solo le colonne rilevanti e rinominandole per maggiore leggibilità.
2. Si costruisce una variabile dummy `Possiede EV`, pari a 1 se l’individuo dichiara di possedere almeno un veicolo elettrico.
3. Si ricodifica la variabile `Formazione scolastica` (`G5`) in una scala ordinale da 1 a 5, utile per analisi quantitative.
4. Poiché il file contiene righe duplicate per alcuni ID (formato long), si aggregano i dati mantenendo una sola riga per individuo. In caso di duplicati di individui che hanno sia un veicolo elettrico sia un veicolo non elettrico, si considera l'individuo come possessore di EV. NOTA: in questo specifico, dopo che viene eseguita l'operazione di rimozione di righe multiple, ad ogni ID corrisponde una sola riga, non so se ci sono casi in cui questo non capita (credo di sì)


In [46]:
import pandas as pd

# FASE 1: Caricamento con selezione e rinomina colonne
mappa_colonne = {
    "ID": "ID",
    "G1": "Genere",
    "Area": "Area geografica",
    "G5": "Formazione scolastica",
    "TA7": "Tipo veicolo posseduto",
    "AMB_tot": "Preoccupazioni ambientali",
    "ATE_1": "Acquisto EV (ATE)",
    "WTP2_1": "WTP EV (%)"
}

# Carica solo le colonne di interesse e rinominale
df_original = pd.read_csv("data/DF.csv", sep=";", encoding="utf-8", usecols=mappa_colonne.keys())
df = df_original.rename(columns=mappa_colonne)

# FASE 2: Preparazione variabili chiave

# Crea dummy "Possiede EV" se il veicolo è elettrico
df["Possiede EV"] = df["Tipo veicolo posseduto"].str.lower().str.contains("elettrico", na=False).astype(int)
df = df.drop("Tipo veicolo posseduto", axis=1)

# Visualizza livelli unici di formazione scolastica

# Dizionario di mappatura dei livelli istruzione
mappa_istruzione = {
    "Nessuna qualifica": 1,
    "Istruzione primaria": 2,
    "Licenza elementare/media": 2,
    "Istruzione secondaria": 3,
    "Diploma di scuola superiore": 3,
    "Istruzione terziaria o superiore": 5,
    "Laurea o titolo superiore": 5
}

# Crea una nuova colonna con i livelli ricodificati
df["Livello istruzione (1-5)"] = df["Formazione scolastica"].map(mappa_istruzione)
df = df.drop("Formazione scolastica", axis=1)

# Visualizza la distribuzione
print("\nDistribuzione livelli di istruzione:")


# Visualizza valori unici per Genere e Area
print("\nValori unici di Genere:", df["Genere"].dropna().unique())
print("Valori unici di Area geografica:", df["Area geografica"].dropna().unique())




# 1. Isola il nome delle colonne da confrontare (tutte tranne "Possiede EV")
colonne_confronto = [col for col in df.columns if col != "Possiede EV"]

# 2. Trova gli ID duplicati che hanno righe identiche nelle altre colonne
duplicati_validi = (
    df.groupby("ID")
    .filter(lambda x: len(x) > 1 and x[colonne_confronto].nunique().eq(1).all())
)

# 3. Ottieni gli ID da filtrare (quelli che soddisfano la condizione)
id_da_filtrare = duplicati_validi["ID"].unique()

# 4. Per questi ID, tieni solo la riga con Possiede EV = 1
righe_filtrate = (
    df[df["ID"].isin(id_da_filtrare)]
    .sort_values(by="Possiede EV", ascending=False)
    .drop_duplicates(subset="ID", keep="first")
)

# 5. Ricostruisci il DataFrame finale:
#    - mantieni tutte le righe che non erano tra i duplicati da filtrare
#    - aggiungi solo le righe filtrate corrette

print(f"Numero di righe PRIMA la rimozione dei duplicati: {len(df)}") 
df = pd.concat([
    df[~df["ID"].isin(id_da_filtrare)],
    righe_filtrate
], ignore_index=True)

print(f"Numero di righe DOPO la rimozione dei duplicati: {len(df)}")


# Mostra le prime righe delle colonne chiave per verifica
print("\nPrime righe del dataset filtrato:")
print(df.head())


print(df["Livello istruzione (1-5)"].value_counts().sort_index())
print(df["Area geografica"].value_counts().sort_index())
print(df["Genere"].value_counts().sort_index())
print(df["Possiede EV"].value_counts().sort_index())




Distribuzione livelli di istruzione:

Valori unici di Genere: ['Femmina' 'Maschio' 'Non binario']
Valori unici di Area geografica: ['Centro' 'Nord' 'Sud e Isole']
Numero di righe PRIMA la rimozione dei duplicati: 3787
Numero di righe DOPO la rimozione dei duplicati: 2001

Prime righe del dataset filtrato:
   ID   Genere Area geografica  Preoccupazioni ambientali  Acquisto EV (ATE)  \
0   2  Maschio            Nord                         55                  1   
1   3  Maschio          Centro                        123                  2   
2   6  Femmina            Nord                        128                  1   
3   8  Femmina          Centro                        121                  2   
4   9  Maschio            Nord                         86                  1   

   WTP EV (%)  Possiede EV  Livello istruzione (1-5)  
0           1            0                         3  
1           1            0                         5  
2           2            0                    

## Codifica delle variabili categoriche: Genere e Area geografica

In questa fase si procede alla trasformazione delle variabili categoriche nominali (Genere e Area geografica) in variabili dummy, necessarie per l’inclusione nel modello di regressione lineare.

### ✳️ Obiettivo

Le variabili categoriche (es. "Maschio", "Femmina", "Nord", "Centro", ecc.) non possono essere usate direttamente in un modello lineare.  
Devono essere convertite in variabili numeriche binarie (0/1), dette dummy.

### ⚙️ Cosa fa il codice

1. **Copia il DataFrame originale** in una nuova variabile `df_dummies` per lavorare in sicurezza.
2. **Crea variabili dummy** per:
   - `Genere` → crea colonne come `Genere_Maschio`, `Genere_Non binario`
   - `Area geografica` → crea colonne come `Area geografica_Nord`, `Area geografica_Sud e Isole`
   - La prima categoria alfabetica di ciascuna variabile viene esclusa automaticamente (`drop_first=True`) per evitare la dummy variable trap.
3. **Stampa i nomi delle nuove colonne dummy** per verifica.
4. **Verifica che il numero di righe e ID sia invariato**, cioè che non siano stati introdotti duplicati o persi dati.
5. **Rimuove le colonne originali** (testuali) perché ormai sono ridondanti.
6. **Visualizza le prime righe del nuovo DataFrame** codificato.

### ✅ Risultato

Il dataset risultante contiene:
- solo variabili numeriche
- le informazioni categoriche codificate come 0/1
- una struttura compatibile con i modelli di regressione


## 

In [47]:
# FASE 4.1: Codifica delle variabili categoriche nominali (Genere, Area geografica)

# Creiamo una copia per sicurezza
df_dummies = df.copy()

# Creazione di variabili dummy, escludendo la prima categoria alfabetica per evitare la dummy variable trap
df_dummies = pd.get_dummies(df_dummies, columns=["Genere", "Area geografica"], drop_first=True)

# Visualizza le nuove colonne create
print("\nColonne create con la codifica dummy:")
print([col for col in df_dummies.columns if "Genere_" in col or "Area geografica_" in col])

# Controlla che il numero di righe e ID sia invariato
print("\nNumero di righe:", len(df_dummies))
print("Numero di ID unici:", df_dummies["ID"].nunique())

# Rimozione delle colonne originali (ormai ridondanti)
df_dummies = df_dummies.drop(columns=["Genere", "Area geografica"], errors="ignore")

# Visualizza le prime righe per verifica
print("\nPrime righe del dataset codificato:")
print(df_dummies.head())


Colonne create con la codifica dummy:
['Genere_Maschio', 'Genere_Non binario', 'Area geografica_Nord', 'Area geografica_Sud e Isole']

Numero di righe: 2001
Numero di ID unici: 2001

Prime righe del dataset codificato:
   ID  Preoccupazioni ambientali  Acquisto EV (ATE)  WTP EV (%)  Possiede EV  \
0   2                         55                  1           1            0   
1   3                        123                  2           1            0   
2   6                        128                  1           2            0   
3   8                        121                  2           1            0   
4   9                         86                  1           1            0   

   Livello istruzione (1-5)  Genere_Maschio  Genere_Non binario  \
0                         3            True               False   
1                         5            True               False   
2                         3           False               False   
3                         3    

## 📈 FASE 4.2 – Preparazione finale del dataset e stima del modello OLS

Questa fase comprende la preparazione delle variabili per il modello di regressione lineare (OLS) e la stima vera e propria.

---

### 🛠️ 1. Rinomina delle colonne

Le colonne con spazi o caratteri speciali (es. `"WTP EV (%)"` o `"Possiede EV"`) vengono rinominate per renderle compatibili con `statsmodels.formula.api`, che richiede nomi semplici e validi nelle formule.  
Esempi:
- `"WTP EV (%)"` → `WTP_EV`
- `"Genere_Maschio"` → `Genere_M`

---

### 🧮 2. Centering delle variabili principali

Per migliorare la stabilità numerica e interpretativa del modello, vengono centrate (mean-centered) le seguenti variabili:
- `Acquisto EV (ATE)` → `ATE_c`
- `Preoccupazioni ambientali` → `AMB_c`
- `Livello istruzione (1-5)` → `ISTR_c`

📌 Il centering consente di interpretare l'intercetta del modello come il valore atteso di WTP_EV per un individuo medio.

---

### ✳️ 3. Costruzione dei termini di interazione

Per verificare se l’istruzione modera l’effetto delle variabili principali (ATE, AMB), si costruiscono due interazioni:
- `ATE × Istruzione` → `ATE_x_ISTR`
- `AMB × Istruzione` → `AMB_x_ISTR`

---

### 🧾 4. Specifica del modello OLS

Viene definita una formula per il modello lineare, che include:
- le variabili principali (`ATE_c`, `AMB_c`)
- il moderatore (`ISTR_c`)
- le due interazioni
- i controlli socio-demografici (dummy per Genere, Area e Possesso EV)

La formula specificata è:

```python
WTP_EV ~ ATE_c + AMB_c + ISTR_c + ATE_x_ISTR + AMB_x_ISTR + Genere_M + Genere_NB + Area_Nord + Area_SudIsole + Possiede_EV


In [48]:
import statsmodels.formula.api as smf

# Rinomina delle colonne con nomi compatibili per statsmodels
df_dummies = df_dummies.rename(columns={
    "WTP EV (%)": "WTP_EV",
    "Possiede EV": "Possiede_EV",
    "Genere_Maschio": "Genere_M",
    "Genere_Non binario": "Genere_NB",
    "Area geografica_Nord": "Area_Nord",
    "Area geografica_Sud e Isole": "Area_SudIsole"
})

# Centering delle variabili principali e del moderatore
df_dummies["ATE_c"] = df_dummies["Acquisto EV (ATE)"] - df_dummies["Acquisto EV (ATE)"].mean()
df_dummies["AMB_c"] = df_dummies["Preoccupazioni ambientali"] - df_dummies["Preoccupazioni ambientali"].mean()
df_dummies["ISTR_c"] = df_dummies["Livello istruzione (1-5)"] - df_dummies["Livello istruzione (1-5)"].mean()

# Creazione dei termini di interazione
df_dummies["ATE_x_ISTR"] = df_dummies["ATE_c"] * df_dummies["ISTR_c"]
df_dummies["AMB_x_ISTR"] = df_dummies["AMB_c"] * df_dummies["ISTR_c"]

# Definizione della formula del modello OLS
formula = (
    "WTP_EV ~ ATE_c + AMB_c + ISTR_c + "
    "ATE_x_ISTR + AMB_x_ISTR + "
    "Genere_M + Genere_NB + Area_Nord + Area_SudIsole + Possiede_EV"
)

# Stima del modello
model = smf.ols(formula=formula, data=df_dummies).fit()

# Visualizzazione dei risultati
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 WTP_EV   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     34.07
Date:                mer, 23 lug 2025   Prob (F-statistic):           9.78e-62
Time:                        17:55:24   Log-Likelihood:                -9347.2
No. Observations:                2001   AIC:                         1.872e+04
Df Residuals:                    1990   BIC:                         1.878e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                34.25

# 📊 Commento ai risultati della regressione OLS

La regressione stima l’effetto di diverse variabili (atteggiamenti, istruzione, fattori socio-demografici) sulla willingness to pay per un veicolo elettrico (`WTP_EV`).

---

## 🎯 Statistiche generali del modello

| Statistica       | Valore    | Interpretazione                                                                 |
|------------------|-----------|----------------------------------------------------------------------------------|
| R²               | 0.146     | Il modello spiega il 14.6% della variabilità di WTP_EV                          |
| Adj. R²          | 0.142     | Molto vicino a R² → il modello è parsimonioso                                  |
| F-statistic      | 34.07     | Il modello è globalmente significativo (p < 0.001)                              |
| N osservazioni   | 2001      | Ampio campione                                                                   |

✅ Il modello è valido nel complesso, anche se la capacità esplicativa è moderata.

---

## 🔹 Variabili principali

| Variabile | Coeff. | p-value | Interpretazione |
|----------|--------|---------|------------------|
| `ATE_c`  | +7.53  | 0.000   | ✔️ Significativa: un atteggiamento più favorevole all'acquisto EV aumenta la WTP |
| `AMB_c`  | +0.01  | 0.691   | ❌ Non significativa: la coscienza ambientale non ha effetto diretto sulla WTP    |

---

## 🧠 Moderatore e interazioni

| Variabile         | Coeff. | p-value | Interpretazione |
|------------------|--------|---------|------------------|
| `ISTR_c`         | –0.66  | 0.246   | ❌ Non significativa: l’istruzione da sola non spiega la WTP |
| `ATE × ISTR`     | –0.07  | 0.883   | ❌ Nessun effetto moderatore sull'atteggiamento |
| `AMB × ISTR`     | –0.03  | 0.158   | ❌ Debole e non significativo |

ℹ️ Il livello di istruzione non risulta essere un moderatore significativo.

---

## 👤 Variabili di controllo

| Variabile            | Coeff. | p-value | Interpretazione |
|----------------------|--------|---------|------------------|
| `Genere_M`           | –3.63  | 0.002   | ✔️ Gli uomini hanno una WTP inferiore di circa 3.6 punti rispetto alle donne |
| `Genere_Non binario` | +8.35  | 0.397   | ❌ Non significativo (ampio intervallo di confidenza) |
| `Area_Nord`          | –0.91  | 0.562   | ❌ Nessuna differenza rispetto al Centro |
| `Area_SudIsole`      | +4.54  | 0.006   | ✔️ Significativo: nel Sud e Isole la WTP è più alta |
| `Possiede_EV`        | +9.00  | 0.007   | ✔️ Chi possiede già un EV è disposto a pagare di più |

---

## Conclusione

- L’atteggiamento personale verso i veicoli elettrici è il fattore più influente sulla willingness to pay.
- La coscienza ambientale e il livello d’istruzione non mostrano effetti diretti né moderatori.
- Tra i controlli, risultano rilevanti: genere, possesso EV, e residenza in Sud/Isole.

📌 Il modello è utile per identificare fattori individuali che influenzano l'accettazione economica dei veicoli elettrici, anche se non esaurisce tutta la variabilità nei dati.

---

## 🔍 Analisi della multicollinearità (VIF)

Per verificare la presenza di multicollinearità tra le variabili indipendenti nel modello, è stato calcolato il Variance Inflation Factor (VIF) per ciascun predittore.

Il VIF misura quanto una variabile è linearmente correlata con le altre variabili del modello.  
Valori elevati di VIF indicano potenziale collinearità, che può rendere instabili le stime dei coefficienti.

### 📐 Interpretazione dei VIF

| VIF         | Interpretazione                                 |
|-------------|--------------------------------------------------|
| < 5         | Nessuna o bassa collinearità → OK                |
| 5–10        | Collinearità moderata → attenzione               |
| > 10        | Alta collinearità → possibile problema serio     |

In [49]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Seleziona le variabili indipendenti usate nel modello
X = df_dummies[[
    "ATE_c", "AMB_c", "ISTR_c", "ATE_x_ISTR", "AMB_x_ISTR",
    "Genere_M", "Genere_NB", "Area_Nord", "Area_SudIsole", "Possiede_EV"
]]

# Aggiungi la costante (intercetta)
X = add_constant(X)

# Converte tutte le colonne in float per evitare problemi
X = X.astype(float)

# Calcolo VIF
vif = pd.DataFrame()
vif["Variabile"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Mostra il risultato
print(vif)

        Variabile       VIF
0           const  6.317808
1           ATE_c  1.229457
2           AMB_c  1.204873
3          ISTR_c  1.072788
4      ATE_x_ISTR  1.167666
5      AMB_x_ISTR  1.182955
6        Genere_M  1.026308
7       Genere_NB  1.006053
8       Area_Nord  1.837414
9   Area_SudIsole  1.823086
10    Possiede_EV  1.046402




### Risultati ottenuti

| Variabile         | VIF   | Interpretazione                       |
|-------------------|-------|----------------------------------------|
| const             | 6.32  | ✅ Accettabile (intercetta centrata)   |
| ATE_c             | 1.23  | ✅ Nessuna collinearità                |
| AMB_c             | 1.20  | ✅ Nessuna collinearità                |
| ISTR_c            | 1.07  | ✅ Nessuna collinearità                |
| ATE_x_ISTR        | 1.17  | ✅ Nessuna collinearità                |
| AMB_x_ISTR        | 1.18  | ✅ Nessuna collinearità                |
| Genere_M          | 1.03  | ✅ Nessuna collinearità                |
| Genere_NB         | 1.01  | ✅ Nessuna collinearità                |
| Area_Nord         | 1.84  | ✅ Nessuna collinearità                |
| Area_SudIsole     | 1.82  | ✅ Nessuna collinearità                |
| Possiede_EV       | 1.05  | ✅ Nessuna collinearità                |

### Conclusione

Tutti i valori di VIF sono ben al di sotto della soglia critica di 5.  
Non si rileva alcuna collinearità significativa tra le variabili indipendenti → il modello è ben specificato da questo punto di vista.

L’inclusione di variabili interattive, dummy e variabili centrate non ha generato problemi di instabilità tra i predittori.
